In [1]:
# 1. Install dependencies
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117
!pip install -q transformers>=4.51.0 sentence-transformers>=2.7.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.4 MB/s eta 0:00:00


In [2]:
# 2. Connect to Google Drive 
from google.colab import drive
drive.mount('/content/drive')
# storage path
EMBEDS_DIR = '/content/drive/MyDrive/ml100k_embeddings'
import os
os.makedirs(EMBEDS_DIR, exist_ok=True)


Mounted at /content/drive


In [3]:
# 3. Load MovieLens100k dataset and extract titles 
import pandas as pd

# download and extract
import urllib.request, zipfile
if not os.path.exists('ml-100k'):
    urllib.request.urlretrieve(
        "https://files.grouplens.org/datasets/movielens/ml-100k.zip",
        "ml-100k.zip"
    )
    with zipfile.ZipFile("ml-100k.zip","r") as z: z.extractall(".")
    os.remove("ml-100k.zip")

# Load items table
items = pd.read_csv(
    "ml-100k/u.item",
    sep="|", encoding="latin-1",
    names=["item_id","title","release","video","imdb"] + [f"g{i}" for i in range(19)]
)

titles = items[['item_id','title']].copy()
print(f"تعداد آیتم‌ها: {len(titles)}")
titles.head()


تعداد آیتم‌ها: 1682


,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [5]:
# 4. Load Qwen3-Embedding-0.6B model
from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B",
    device="cuda",  
    tokenizer_kwargs={"padding_side": "left"}
)

# Verify embedding dimensions
EMB_DIM = model.get_sentence_embedding_dimension()
print(f"Embedding dimension: {EMB_DIM}")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

Embedding dimension: 1024


In [6]:
# 5. Compute embeddings for titles 
# Convert the list of titles to a list of strings
docs = titles['title'].tolist()

# Batch encoding
batch_size = 64
embeddings = []
for i in range(0, len(docs), batch_size):
    batch = docs[i:i+batch_size]
    embs = model.encode(batch, convert_to_numpy=True, show_progress_bar=True)
    embeddings.append(embs)

import numpy as np
embeddings = np.vstack(embeddings)
print("Shape of embeddings:", embeddings.shape)


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Shape of embeddings: (1682, 1024)


In [7]:
# 6. Save embeddings 
# Numpy file
np.save(os.path.join(EMBEDS_DIR, 'item_title_embeddings.npy'), embeddings)

# create a DataFrame
df_emb = pd.DataFrame(
    embeddings,
    index=titles['item_id'],
    columns=[f"dim_{i}" for i in range(embeddings.shape[1])]
)
df_emb.to_csv(os.path.join(EMBEDS_DIR, 'item_title_embeddings.csv'))

print("Embeddings saved to:", EMBEDS_DIR)


Embeddings saved to: /content/drive/MyDrive/ml100k_embeddings
